In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model_name = 'jbl.keras'
model = load_model(model_name)

In [ ]:
import os
import time
import uuid
import cv2
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = model.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    print("yhat[0] = ", yhat)
    if yhat[0] > 0.1: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, model_name, tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('Webcam', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
# def load_image(x): 
#     byte_img = tf.io.read_file(x)
#     img = tf.io.decode_jpeg(byte_img)
#     return img
    
# new_images = tf.data.Dataset.list_files('old\\data\\train\\images\\*.jpg', shuffle=False)
# new_images = new_images.map(load_image)
# new_images = new_images.map(lambda x: tf.image.resize(x, (120,120)))
# new_images = new_images.map(lambda x: x/255)

# test = tf.data.Dataset.zip((new_images, ()))
# test = test.shuffle(1300)
# test = test.batch(8)
# test = test.prefetch(4)

In [ ]:
# test_data = test.as_numpy_iterator()
# test_sample = test_data.next()
# # print(test_sample)
# yhat = model.predict(test_sample)
# print("yhat[0] = ", yhat)
# fig, ax = plt.subplots(ncols=8, figsize=(20,20))
# for idx in range(8): 
#     sample_image = test_sample[0][idx].copy()
#     sample_coords = yhat[1][idx]
    
#     if yhat[0][idx] > 0.9:
#         cv2.rectangle(sample_image, 
#                       tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
#                       tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)), 
#                             (255,0,0), 2)
    
#     ax[idx].imshow(sample_image)

In [ ]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img
    
new_images = tf.data.Dataset.list_files('data\\augmented_data\\test\\images\\*.jpg', shuffle=False)
new_images = new_images.map(load_image)
new_images = new_images.map(lambda x: tf.image.resize(x, (120,120)))
new_images = new_images.map(lambda x: x/255)

test = tf.data.Dataset.zip((new_images, ()))
test = test.shuffle(1300)
test = test.batch(8)
test = test.prefetch(4)


test_data = test.as_numpy_iterator()
test_sample = test_data.next()
yhat = model.predict(test_sample[0])
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx].copy()
    sample_coords = yhat[1][idx]
    
    if yhat[0][idx] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)), 
                            (255,0,0), 2)
    
    ax[idx].imshow(sample_image)